# FAIRHealth PHT PPDML

In [1]:
import requests
import json
import uuid
import time
signalStation = "http://vwdata.personalhealthtrain.net"

def getStations(signalStation):
    response = requests.get(signalStation + "/client")
    return json.loads(response.text)

def defineTrain(image, inputVars, runId):
    return {
        "image": image,
        "inputString": ("%s" % json.dumps(inputVars)),
        "runId": runId
    }

def requestExecution(signalStation, clientId, taskDescription):
    response = requests.post(signalStation + "/client/" + str(clientId) + "/task/add", json=taskDescription)
    taskId = json.loads(response.text)["taskId"]
    return taskId

def getExecutionResult(signalStation, clientId, taskId, wait=True):
    url = signalStation + "/client/" + str(clientId) + "/task/" + str(taskId) + "/result"
    jsonResult = json.loads(requests.get(url).text)
    
    while len(jsonResult)==0 & wait:
        time.sleep(2)
        jsonResult = json.loads(requests.get(url).text)
    
    return jsonResult

## List Stations

In [2]:
getStations(signalStation)

[{'id': 1,
  'name': 'The Maastricht Study',
  'email': 'johan.vansoest@maastro.nl',
  'institute': 'Maastricht University',
  'country': 'The Netherlands',
  'ip': '192.168.96.1',
  'last_seen': '2019-02-24 10:29:30.436317'},
 {'id': 2,
  'name': 'VWData Station Maastro',
  'email': 'johan.vansoest@maastro.nl',
  'institute': 'MAASTRO Clinic',
  'country': 'Netherlands',
  'ip': '192.168.96.1',
  'last_seen': '2019-02-24 10:29:00.309839'}]

## Stage 1 at Maastricht Study

### Define train

In [3]:
inputData = {
    "stage": 1
}
myTrain = defineTrain("fairhealth/ppdml", inputData, 1)

### Run on client

In [4]:
clientId = 1
taskId = requestExecution(signalStation, clientId, myTrain)
print("Task submitted with ID: %s" % taskId)

Task submitted with ID: 17


### Receive result

In [7]:
response = requests.get(signalStation + "/client/" + str(clientId) + "/task/" + str(taskId) + "/result")
results = getExecutionResult(signalStation, clientId, taskId)
print(results[0]["log"])
result = json.loads(results[0]["response"])
print(result.keys())

Stage: 1


=======================INTERNAL FILE LOG===================== 

dict_keys(['matrixBytes', 'sumNoiseBytes'])


## Stage 2 at CBS/Maastro endpoint

In [ ]:
clientId = 2

## TODO